In [ ]:
import numpy as np
import os
import json

In [ ]:
# Loading train/test set downloaded from FEVER website: Note all instances of 'dev' below refer to the test set:

os.chdir('/Users/brandoncheong/Desktop/ML_Dissertation/wiki-pages')
train_data = list(np.load('train_data.npy', allow_pickle = 'True'))
dev_data = list(np.load('dev_data.npy', allow_pickle = 'True'))

In [ ]:
# Loading Wikipedia repository:
files1 = ['wiki-00' + str(i) + '.jsonl' for i in range(1, 10)]
files2 = ['wiki-0' + str(i) + '.jsonl' for i in range(10, 100)]
files3 = ['wiki-' + str(i) + '.jsonl' for i in range(100, 110)]

files = files1 + files2 + files3

wiki_repo = list()
for file in files:
    with open(file, 'r') as file: 
         wiki_repo += [json.loads(string.replace('""','\""')) for string in file]

In [ ]:
# train data processing:
train_evidence_statements = {}
train_data_cleaned = []


for dictionary in train_data:
    dictionary = json.loads(dictionary)
    articles = list()
    for entry in dictionary['evidence']:
            for subentry in entry:
                article = subentry[2]
                articles.append(article)
    articles = set(articles)
    for article in articles: 
           if article not in list(train_evidence_statements.keys()):
            train_evidence_statements[article] = [item['text'] for item in wiki_repo if item['id'] == article]
            dictionary['evidence'].append(train_evidence_statements[article])
           else:
            dictionary['evidence'] = list(train_evidence_statements)
            dictionary['evidence'].append(train_evidence_statements[article])
    train_data_cleaned.append(dictionary)

In [ ]:
# same for dev data:
dev_evidence_statements = {}
dev_data_cleaned = []


for dictionary in dev_data:
    dictionary = json.loads(dictionary)
    articles = list()
    for entry in dictionary['evidence']:
            for subentry in entry:
                article = subentry[2]
                articles.append(article)
    articles = set(articles)
    for article in articles: 
           if article not in list(dev_evidence_statements.keys()):
            dev_evidence_statements[article] = [item['text'] for item in wiki_repo if item['id'] == article]
            dictionary['evidence'].append(dev_evidence_statements[article])
           else:
            dictionary['evidence'] = list()
            dictionary['evidence'].append(dev_evidence_statements[article])
    dev_data_cleaned.append(dictionary)

In [ ]:
print(dev_data_cleaned[2])
print(len(dev_data_cleaned))

In [ ]:
full_train_data_new = list() 

for dictionary in train_data_cleaned:
   if dictionary['label'] == 'NOT ENOUGH INFO':
      alt_dictionary = copy.deepcopy(dictionary)
      alt_dictionary['evidence'] = ' '  
      full_train_data_new.append(alt_dictionary)

   
   else:
    
 
     for element in dictionary['evidence']:
        
        
        
        if len(element) == 0 and dictionary['evidence'].index(element) != len(dictionary['evidence']) - 1:

            continue
            
        elif len(element) == 0 and dictionary['evidence'].index(element) == len(dictionary['evidence']) - 1:
            case = False

            break
            
        elif dictionary['evidence'].index(element) != len(dictionary['evidence']) - 1 and type(element[0]) != str:
            continue
        
            
        
        elif dictionary['evidence'].index(element) == len(dictionary['evidence']) - 1 and type(element[0]) != str:
            case = False

            break
        
        else:
            
            case = True

            idx = dictionary['evidence'].index(element) 
            
    
     if case == True and idx == len(dictionary['evidence']) - 1:
        
            
        text = dictionary['evidence'][idx]        
        
        alt_dictionary = copy.deepcopy(dictionary)
        alt_dictionary['evidence'] = text[0]
        full_train_data_new.append(alt_dictionary)
    
    
     elif case == True and idx < len(dictionary['evidence']) - 1:  
        
        texts = dictionary['evidence'][idx:] # adapt to specific structure of the data: this is a list of lists
        initial_text = texts[0]
        for text in texts[1:]:
            initial_text += text
        
        alt_dictionary = copy.deepcopy(dictionary)

        alt_dictionary['evidence'] = initial_text
        full_train_data_new.append(alt_dictionary)
    
     else: # idx does not exist
        continue        

In [ ]:
full_dev_data_new = list() 

for dictionary in dev_data_cleaned:
   if dictionary['label'] == 'NOT ENOUGH INFO':
      alt_dictionary = copy.deepcopy(dictionary)
      alt_dictionary['evidence'] = ' '  
      full_dev_data_new.append(alt_dictionary)

   
   else:
    
 
     for element in dictionary['evidence']:
        
        
        
        if len(element) == 0 and dictionary['evidence'].index(element) != len(dictionary['evidence']) - 1:

            continue
            
        elif len(element) == 0 and dictionary['evidence'].index(element) == len(dictionary['evidence']) - 1:
            case = False

            break
            
        elif dictionary['evidence'].index(element) != len(dictionary['evidence']) - 1 and type(element[0]) != str:
            continue
        
            
        
        elif dictionary['evidence'].index(element) == len(dictionary['evidence']) - 1 and type(element[0]) != str:
            case = False

            break
        
        else:
            
            case = True

            idx = dictionary['evidence'].index(element) 
            
    
     if case == True and idx == len(dictionary['evidence']) - 1:
        
            
        text = dictionary['evidence'][idx]        
        
        alt_dictionary = copy.deepcopy(dictionary)
        alt_dictionary['evidence'] = text[0]
        full_dev_data_new.append(alt_dictionary)
    
    
     elif case == True and idx < len(dictionary['evidence']) - 1:  
        
        texts = dictionary['evidence'][idx:] # adapt to specific structure of the data: this is a list of lists
        initial_text = texts[0]
        for text in texts[1:]:
            initial_text += text
        
        alt_dictionary = copy.deepcopy(dictionary)

        alt_dictionary['evidence'] = initial_text
        full_dev_data_new.append(alt_dictionary)
    
     else: # idx does not exist
        continue        

In [ ]:
# NEI Processing for both Train and Test sets:

from random import seed
from random import choice
# seed random number generator
seed(1)
# prepare a sequence
sequence = [i for i in range(len(wiki_repo[1:]))]
# make choices from the sequence
random_choice_train = [choice(sequence) for i in range(35639)]
print(len(random_choice_train))

# seed random number generator
# prepare a sequence
sequence = [i for i in range(len(wiki_repo[1:]))]
# make choices from the sequence
random_choice_dev = [choice(sequence) for i in range(6666)]
print(len(random_choice_dev))

In [ ]:
count = 0 
for dictionary in full_train_data_new:
   if dictionary['label'] == 'NOT ENOUGH INFO':
      idx = full_train_data_new.index(dictionary)
      wiki_idx = random_choice_train[count] 
      alt_dictionary = copy.deepcopy(dictionary)
      alt_dictionary['evidence'] = wiki_repo[wiki_idx + 1]['text'] 
      full_train_data_new[idx] = alt_dictionary
      count += 1
   else:
        continue

In [ ]:
count = 0 
for dictionary in full_dev_data_new:
   if dictionary['label'] == 'NOT ENOUGH INFO':
      idx = full_dev_data_new.index(dictionary)
      wiki_idx = random_choice_dev[count] 
      alt_dictionary = copy.deepcopy(dictionary)
      alt_dictionary['evidence'] = wiki_repo[wiki_idx + 1]['text'] 
      full_dev_data_new[idx] = alt_dictionary
      count += 1
   else:
        continue

In [ ]:
np.save('full_train_data_new.npy', full_train_data_new)
np.save('full_dev_data_new.npy', full_dev_data_new)